In [1]:
from ctapipe.core import Tool, Component
from traitlets import (Integer, Float, List, Dict,Unicode, TraitError, observe)
import logging
from time import sleep

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

In [2]:
class MyComponent(Component):
    description = "Do some things"

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")

# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    name="AdvancedComponent"
    description = "something more advanced"

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Unicode(help="input file name").tag(config=True)
    outfile = Unicode(help="output file name").tag(config=True)

    @observe('outfile')
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))


class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = Dict(dict(infile='AdvancedComponent.infile',
                        iterations='MyTool.iterations'))

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def init_comp(self):
        self.comp = MyComponent(self, config=self.config)
        self.comp2 = SecondaryMyComponent(self, config=self.config)

    def init_advanced(self):
        self.advanced = AdvancedComponent(self, config=self.config)

    def initialize(self):
        self.init_comp()
        self.init_advanced()

    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.5)
            
    def finish(self):
        self.log.warning("Shutting down.")
    


In [3]:
tool=MyTool()

In [4]:
tool.print_help()

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--log_level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--infile=<Unicode> (AdvancedComponent.infile)
    Default: ''
    input file name

To see all available configurables, use `--help-all`



In [5]:
tool.print_help(classes=True)

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--log_level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--infile=<Unicode> (AdvancedComponent.infile)
    Default: ''
    input file name

Class parameters
----------------

Parameters are set from command-line arguments of the form:
`--Class.trait=value`. This line is evaluated in Python, so simple expressions
are allowed, e.g.:: `--C.a='range(3)'` For setting C.a=[0,1,2].

MyTool options
--------------
--MyTool.config_file=<Unicode>
    Default: ''
    name of configuration file with parameters
--MyTool.iterations=<Int>
    Default: 5
    Number of time

In [6]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 
tool.run(argv=['--log_level','INFO','--infile','bork.txt','--iterations','3'])

2016-06-27 14:32:30 : INFO [MyTool run] Starting: mytool
2016-06-27 14:32:30 : INFO [MyTool start] Performing 3 iterations...
2016-06-27 14:32:30 : INFO [MyTool start] ITERATION 0
2016-06-27 14:32:30 : INFO [MyTool start] ITERATION 1
2016-06-27 14:32:31 : INFO [MyTool start] ITERATION 2
2016-06-27 14:32:31 : WARNING [MyTool finish] Shutting down.


In [12]:
tool.run(argv=['--log_level','DEBUG','--infile','bork.txt'])

2016-06-24 19:39:33 : INFO [MyTool run] Starting: mytool
2016-06-24 19:39:33 : DEBUG [MyTool run] CONFIG: {'AdvancedComponent': {'infile': 'bork.txt'}, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}
2016-06-24 19:39:33 : INFO [MyTool start] Performing 3 iterations...
2016-06-24 19:39:33 : INFO [MyTool start] ITERATION 0
2016-06-24 19:39:33 : DEBUG [MyTool.MyComponent do_thing] Did thing
2016-06-24 19:39:33 : DEBUG [MyTool.SecondaryMyComponent do_thing] Did thing
2016-06-24 19:39:34 : INFO [MyTool start] ITERATION 1
2016-06-24 19:39:34 : DEBUG [MyTool.MyComponent do_thing] Did thing
2016-06-24 19:39:34 : DEBUG [MyTool.SecondaryMyComponent do_thing] Did thing
2016-06-24 19:39:34 : INFO [MyTool start] ITERATION 2
2016-06-24 19:39:34 : DEBUG [MyTool.MyComponent do_thing] Did thing
2016-06-24 19:39:34 : DEBUG [MyTool.SecondaryMyComponent do_thing] Did thing
2016-06-24 19:39:35 : WARNING [MyTool finish] Shutting down.


you can also set parameters directly in the class

In [11]:
tool.iterations = 1
tool.log_level = 0
tool.run('')

2016-06-27 14:34:11 : WARNING [MyTool finish] Shutting down.


In [12]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)

bad value: The 'iterations' trait of a MyTool instance must be an int, but a value of 'badval' <class 'str'> was specified.


Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [13]:
tool.advanced.outfile = "Another.txt"

2016-06-27 14:34:17 : WARNING [MyTool.AdvancedComponent on_outfile_changed] Outfile was changed to '{'owner': <__main__.AdvancedComponent object at 0x10d7f8710>, 'old': '', 'name': 'outfile', 'type': 'change', 'new': 'Another.txt'}'


In [14]:
from traitlets.config import Application

In [15]:
Application??

In [16]:
print(tool.config)

{'MyTool': {'iterations': 3, 'log_level': 'INFO'}, 'AdvancedComponent': {'infile': 'bork.txt'}}
